![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Etapa 1: Instalação das dependências

In [1]:
!pip uninstall -y tensorflow #Comando necessário, pois o TensorFlow-gpu não desinstala a versão mais recente do Tensorflow, pode gerar conflitos.

Uninstalling tensorflow-2.2.0:
  Successfully uninstalled tensorflow-2.2.0


In [2]:
!pip install tensorflow-gpu==2.0

     |████████████████████████████████| 380.8MB 46kB/s 
     |████████████████████████████████| 450kB 42.5MB/s 
     |████████████████████████████████| 3.8MB 39.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=77d73f68cbc76bc548b9251361f18377173893cea1b00fed1a023faf01f621ac
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2


In [3]:
!pip install tqdm

### Fazendo o download da base de dados de gatos e cachorros

In [4]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-06-22 18:03:37--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 2404:6800:4008:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  84.7MB/s    in 0.8s    

2020-06-22 18:03:38 (84.7 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Etapa 2: Pré-processamento

### Importação das bibliotecas

In [6]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0'

### Descompactando a base de dados de gatos e cachorros

In [7]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [8]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [9]:
zip_object.extractall("./")

In [10]:
zip_object.close()

### Configurando os caminhos (paths)

In [11]:
dataset_path_new = "./cats_and_dogs_filtered"

In [12]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construindo o modelo

### Carregando o modelo pré-treinado (MobileNetV2)

In [13]:
img_shape = (128, 128, 3)

In [14]:
base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, 
                                               include_top = False,
                                               weights = "imagenet")

9412608/9406464 [==============================] - 2s 0us/step


In [15]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Congelando o modelo base

In [16]:
base_model.trainable = False

### Definindo o cabeçalho personalizado da rede neural

In [17]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [19]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [20]:
global_average_layer

<tf.Tensor 'global_average_pooling2d_1/Identity:0' shape=(None, 1280) dtype=float32>

In [21]:
prediction_layer = tf.keras.layers.Dense(units = 1, activation = "sigmoid")(global_average_layer)

### Definindo o modelo

In [22]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

### Compilando o modelo

In [24]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001),
              loss="binary_crossentropy", metrics = ["accuracy"])

### Criando geradores de dados (Data Generators)

Redimensionando as imagens

    Grandes arquiteturas treinadas suportam somente alguns tamanhos pré-definidos.

Por exemplo: MobileNet (que estamos usando) suporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [25]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [26]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")
valid_generator = data_gen_train.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Treinando o modelo

In [27]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 19s 1s/step - loss: 0.7667 - accuracy: 0.5200 - val_loss: 0.6254 - val_accuracy: 0.6640
Epoch 2/5
16/16 [==============================] - 13s 794ms/step - loss: 0.6904 - accuracy: 0.5875 - val_loss: 0.5606 - val_accuracy: 0.7090
Epoch 3/5
16/16 [==============================] - 13s 791ms/step - loss: 0.6394 - accuracy: 0.6295 - val_loss: 0.5071 - val_accuracy: 0.7610
Epoch 4/5
16/16 [==============================] - 13s 800ms/step - loss: 0.5898 - accuracy: 0.6870 - val_loss: 0.4597 - val_accuracy: 0.7910
Epoch 5/5
16/16 [==============================] - 13s 787ms/step - loss: 0.5436 - accuracy: 0.7350 - val_loss: 0.4196 - val_accuracy: 0.8150


### Avaliação do modelo de transferência de aprendizagem

In [28]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [29]:
valid_accuracy

0.815

In [30]:
valid_loss

0.42008210346102715

## Fine tuning


Duas questões principais:

- NÃO USE Fine Tuning em toda a rede neural, pois somente em algumas camadas já é suficiente. A ideia é adotar parte específica da rede neural para nosso problema específico
- Inicie o Fine Tuning DEPOIS que você finalizou a transferência de aprendizagem. Se você tentar o Fine Tuning imediatamente, os gradientes serão muito diferentes entre o cabeçalho personalizado e algumas camadas descongeladas do modelo base 

### Descongelando algumas camadas do topo do modelo base

In [31]:
base_model.trainable = True
len(base_model.layers)

155

In [32]:
fine_tuning_at = 100

In [33]:
for layer in base_model.layers[:fine_tuning_at]:
  layer.trainable = False

### Compilando o modelo para fine tuning

In [34]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Fine tuning

In [35]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 15s 950ms/step - loss: 0.1888 - accuracy: 0.9290 - val_loss: 0.1012 - val_accuracy: 0.9630
Epoch 2/5
16/16 [==============================] - 14s 863ms/step - loss: 0.0276 - accuracy: 0.9975 - val_loss: 0.0851 - val_accuracy: 0.9740
Epoch 3/5
16/16 [==============================] - 14s 859ms/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 0.1034 - val_accuracy: 0.9690
Epoch 4/5
16/16 [==============================] - 14s 854ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.1126 - val_accuracy: 0.9690
Epoch 5/5
16/16 [==============================] - 14s 851ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.1507 - val_accuracy: 0.9640


### Avaliação do modelo com fine tuning

In [36]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [37]:
valid_accuracy

0.964

In [38]:
valid_loss

0.1491578659042716